# AG3
### Github: https://github.com/comandantexd/03MAIR_Algoritmos_optimizacion/blob/master/AG3.ipynb

# Precarga, imports y variables

In [1]:
import urllib
import tsplib95
import random
import math
import copy
import numpy as np

file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Objeto problem
problem = tsplib95.load(file)

#Nodos
nodes = list(problem.get_nodes())

#Aristas
edges = list(problem.get_edges())

# Utilidades

In [2]:
#Genera una colucion aleatoria con comienzo en el nodo 0
def generate_solution(nodes): 
    sol = [nodes[0]]
    for n in nodes[1:]:
        sol = sol + [random.choice(list(set(nodes) - set({nodes[0]}) - set(sol)))]

    return sol

#Distancia entre dos nodos
def distance(a,b, problem):
    return problem.get_weight(a, b)

#Distancia total de una trayectoria/solucion
def total_distance(sol, problem):
    dist = 0
    for i in range(len(sol) - 1):
        dist += distance(sol[i], sol[i + 1], problem)

    return dist + distance(sol[len(sol) - 1], sol[0], problem)

# Búsqueda aleatoria

In [4]:
def random_search(problem, n):
    nodes = list(problem.get_nodes())

    best_sol = []
    best_dist = 10e100

    for i in range(n):
        sol = generate_solution(nodes)
        dist = total_distance(sol, problem)

        if dist < best_dist:
            best_sol = sol
            best_dist = dist
    
    return best_sol, best_dist

sol, dist = random_search(problem, 50000)    

print('Mejor solución: {}'.format(sol))
print('Distancia : {}'.format(dist))
 

Mejor solución: [0, 15, 16, 14, 19, 13, 25, 40, 38, 34, 11, 12, 22, 8, 39, 32, 31, 28, 5, 29, 41, 33, 20, 27, 1, 7, 18, 3, 30, 4, 21, 10, 9, 24, 23, 6, 17, 35, 37, 2, 26, 36]
Distancia : 3591


In [6]:
def generate_neighbor(solution, problem):
    #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones

    best_sol = []
    best_dist = 10e100

    for i in range(1, len(sol) - 1): #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
        for j in range(i + 1, len(sol)): #i+1 por la simetríA
      
            # Generar una nueva solucion intercambaindo nodos
            neighbor = solution[:i] + [solution[j]] + solution[i+1:j] + [solution[i]] + solution[j+1:]

            n_dist = total_distance(neighbor, problem) #Evaluar la solucion

            if n_dist <= best_dist:
                best_dist = n_dist
                best_sol = neighbor

    return best_sol



new_sol = generate_neighbor(sol, problem)

print('Distancia antigua Solucion: {}'.format(total_distance(sol, problem)))
print('Distancia nueva Solucion: {}'.format(total_distance(new_sol, problem)))

Distancia antigua Solucion: 3591
Distancia nueva Solucion: 3376


# Búsqueda local

In [9]:
def local_search(problem, nodes):
    best_sol = []
    #Generar una solucion inicial de referencia(aleatoria)
    sol_ref = generate_solution(nodes)
    best_dist = total_distance(sol_ref, problem)

    finish = False
    iter = 0
    while not finish:
        iter += 1
        neighbor = generate_neighbor(sol_ref, problem)
        n_dist = total_distance(neighbor, problem)

        if n_dist < best_dist:
            best_sol = copy.deepcopy(neighbor)
            best_dist = n_dist
        else:
            print('Iteracion: %d' % iter)
            finish = True #Terminamos, significa que es un mínimo local
        
        sol_ref = neighbor

    return best_sol, best_dist

sol, dist = local_search(problem, nodes)
print('Mejor solución: {}'.format(sol))
print('Distancia total: {}'.format(dist))

Iteracion: 40
Mejor solución: [0, 27, 39, 22, 38, 34, 32, 3, 26, 5, 7, 37, 17, 31, 36, 35, 33, 20, 1, 6, 14, 15, 16, 19, 13, 12, 11, 10, 29, 30, 28, 2, 4, 18, 25, 41, 40, 24, 21, 23, 9, 8]
Distancia total: 1875


In [10]:
#Generador de 1 solucion vecina 2-opt aleatoria (intercambiar 2 nodos)
def generate_random_neighbor(solution):
    #Se eligen dos nodos aleatoriamente
    i, j = sorted(random.sample( range(1, len(solution)), 2))

    #Swap de nodos
    return solution[:i] + [solution[j]] + solution[i+1:j] + [solution[i]] + solution[j+1:]
  
 
print(generate_random_neighbor(sol) )

[0, 27, 39, 22, 38, 34, 32, 3, 26, 5, 24, 37, 17, 31, 36, 35, 33, 20, 1, 6, 14, 15, 16, 19, 13, 12, 11, 10, 29, 30, 28, 2, 4, 18, 25, 41, 40, 7, 21, 23, 9, 8]


# Recocido simulado

In [12]:
#Funcion de probabilidad para aceptar peores soluciones
def probability(T, d):
    return random.random() < math.exp(-1 * d / T)


#Funcion de descenso de temperatura
def lower_temp(T):
    return T * 0.99

    
def simulated_annealing(problem, temp, nodes):
    sol_ref = generate_solution(nodes)
    dist_ref = total_distance(sol_ref, problem)
  
    best_sol = []
    best_dist = 10e100
  
  
    n = 0
    while temp > 0.0001:
        n += 1
        #Genera una solución vecina
        neighbor = generate_random_neighbor(sol_ref)
    
        #Calcula su valor(distancia)
        n_dist = total_distance(neighbor, problem)
      
        #Si es la mejor solución de todas se guarda(siempre!!!)
        if n_dist < best_dist:
            best_sol = neighbor
            best_dist = n_dist
    
        #Si la nueva vecina es mejor se cambia  
        #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
        if n_dist < dist_ref or probability(temp, abs(dist_ref - n_dist)):
            sol_ref = copy.deepcopy(neighbor)
            dist_ref = n_dist

        #Baja temperatura
        temp = lower_temp(temp)


    return best_sol, best_dist

sol, dist  = simulated_annealing(problem, 100000000000, nodes)
 
print('Mejor solución: {}'.format(sol))
print('Distancia total: {}'.format(dist))

Mejor solución: [0, 29, 39, 22, 40, 24, 21, 9, 1, 37, 36, 35, 20, 30, 38, 33, 34, 32, 28, 25, 12, 18, 11, 13, 19, 26, 6, 7, 17, 31, 2, 10, 8, 23, 41, 5, 16, 14, 15, 4, 3, 27]
Distancia total: 2234
